# Практика. API Headhunter

Проведем анализ вакансий для Data Science. Документация API hh: https://api.hh.ru/openapi/redoc

In [2]:
import requests
from bs4 import BeautifulSoup
import json
from tqdm import tqdm
import numpy as np
import pandas as pd
from time import sleep
from collections import Counter

Запросы к API всегда выстраиваются в таком формате: 

URL API + нужный раздел + API-ключ (по требованию) + параметры

https://api.hh.ru/vacancies?text=data%20science&only_with_salary=true&per_page=100

Что в этом запросе?

`https://api.hh.ru/vacancies` – запрос к БД с вакансиями \
`?text=data%20science` – прицепляем с помощью **?** первый параметр `text` с поиском по тексу вакансий \
`&only_with_salary=true` – все следующие параметры прицепляем через **&**. Ищем вакансии только с указанной з/п \
`&per_page=100` – используем пагинацию

In [13]:
vacancies = []

for i in range(0, 2):
    vac = requests.get(f"https://api.hh.ru/vacancies?text=data%20science&only_with_salary=true&per_page=100&page={i}")
    vacancies.extend(json.loads(vac.text)['items'])

In [14]:
vacancies[0]

{'id': '91757292',
 'premium': False,
 'name': 'Junior Data Scientist',
 'department': None,
 'has_test': False,
 'response_letter_required': False,
 'area': {'id': '2',
  'name': 'Санкт-Петербург',
  'url': 'https://api.hh.ru/areas/2'},
 'salary': {'from': None, 'to': 60000, 'currency': 'RUR', 'gross': True},
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': {'city': 'Санкт-Петербург',
  'street': 'набережная Обводного канала',
  'building': '60',
  'lat': 59.9154,
  'lng': 30.342072,
  'description': None,
  'raw': 'Санкт-Петербург, набережная Обводного канала, 60',
  'metro': {'station_name': 'Обводный Канал',
   'line_name': 'Фрунзенско-Приморская',
   'station_id': '18.252',
   'line_id': '18',
   'lat': 59.914686,
   'lng': 30.34815},
  'metro_stations': [{'station_name': 'Обводный Канал',
    'line_name': 'Фрунзенско-Приморская',
    'station_id': '18.252',
    'line_id': '18',
    'lat': 59.914686,
    'lng': 30.34815}],
  'id': '6132717'},
 'response_url': None,
 'sort_

In [15]:
for v in vacancies:
    print(v['name'], v['salary']['from'], v['salary']['to'], v['area']['name'], v['url'])

Junior Data Scientist None 60000 Санкт-Петербург https://api.hh.ru/vacancies/91757292?host=hh.ru
Junior DevOps Engineer 60000 100000 Санкт-Петербург https://api.hh.ru/vacancies/91358751?host=hh.ru
Тестировщик/QA инженер (middle/senior) None 250000 Москва https://api.hh.ru/vacancies/91589923?host=hh.ru
Куратор курсов 50000 50000 Санкт-Петербург https://api.hh.ru/vacancies/91657896?host=hh.ru
Junior Data Scientist None 60000 Новосибирск https://api.hh.ru/vacancies/91901283?host=hh.ru
Специалист-аналитик None 60000 Санкт-Петербург https://api.hh.ru/vacancies/91857388?host=hh.ru
Разработчик Backend None 300000 Москва https://api.hh.ru/vacancies/91589925?host=hh.ru
Специалист технической поддержки (г. Сочи) 170000 234000 Сочи https://api.hh.ru/vacancies/91432199?host=hh.ru
Software Engineer (C#) None 6000 Москва https://api.hh.ru/vacancies/91753968?host=hh.ru
Frontend-разработчик (Angular/React/Vue.js) уровня middle+, senior None 300000 Москва https://api.hh.ru/vacancies/91589927?host=hh.ru

Соберем все ссылки на вакансии, чтобы вытащить оттуда описание вакансии и ключевые навыки.

In [16]:
links = []

for v in vacancies:
    links.append(v['url'])

In [17]:
test = requests.get(links[0])
test0 = json.loads(test.text)
test0

{'id': '91757292',
 'premium': False,
 'billing_type': {'id': 'standard', 'name': 'Стандарт'},
 'relations': [],
 'name': 'Junior Data Scientist',
 'insider_interview': None,
 'response_letter_required': False,
 'area': {'id': '2',
  'name': 'Санкт-Петербург',
  'url': 'https://api.hh.ru/areas/2?host=hh.ru'},
 'salary': {'from': None, 'to': 60000, 'currency': 'RUR', 'gross': True},
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': {'city': 'Санкт-Петербург',
  'street': 'набережная Обводного канала',
  'building': '60',
  'lat': 59.9154,
  'lng': 30.342072,
  'description': None,
  'raw': 'Санкт-Петербург, набережная Обводного канала, 60',
  'metro': {'station_name': 'Обводный Канал',
   'line_name': 'Фрунзенско-Приморская',
   'station_id': '18.252',
   'line_id': '18',
   'lat': 59.914686,
   'lng': 30.34815},
  'metro_stations': [{'station_name': 'Обводный Канал',
    'line_name': 'Фрунзенско-Приморская',
    'station_id': '18.252',
    'line_id': '18',
    'lat': 59.914686,


In [18]:
test0['description']

'<p>В команду аналитиков требуется кандидат, умеющий быстро и точно анализировать большие объемы данных.</p> <p><strong>Вам предстоит заниматься:</strong></p> <ul> <li>Анализом данных с помощью Python - Ad Hoc аналитикой</li> <li>Подготовкой данных и обучением моделей ML</li> <li>Подготовкой ТЗ для команды разработчиков</li> <li>Визуализацией данных, созданием презентаций и отчетности</li> </ul> <strong>Требования:</strong> <ul> <li>Знание Python (pandas, numpy, sklearn, seaborn, matploltlib, plotly)</li> <li>Знание SQL (умение создавать таблицы и выводить данные)</li> <li>Высшее образование (математическое/информационные технологии)</li> <li>Знание ML будут плюсом.</li> </ul> <strong>Условия:</strong> <ul> <li>Стабильный оклад</li> <li>Официальное оформление по ТК РФ с первого дня</li> <li>График работы: 5/2, с 10:00 до 18:30. Суббота, воскресенье-выходные</li> <li>Отдых летом на море и зимой на горнолыжных курортах при финансовой поддержке от группы компаний</li> <li>Компенсация расх

In [19]:
test0['key_skills']

[{'name': 'Python'},
 {'name': 'SQL'},
 {'name': 'Анализ данных'},
 {'name': 'Работа с базами данных'},
 {'name': 'Data Science'},
 {'name': 'Математический анализ'},
 {'name': 'Аналитика'}]

In [20]:
[i['name'] for i in test0['key_skills']]

['Python',
 'SQL',
 'Анализ данных',
 'Работа с базами данных',
 'Data Science',
 'Математический анализ',
 'Аналитика']

In [26]:
json.loads(v.text)

{'errors': [{'value': 'captcha_required',
   'captcha_url': 'https://hh.ru/account/captcha?state=pxvcxBozfu7ry7R4QCetFmUA3SFZr_Pa4yXPQEusAiQg0PPA0K1WhG6WxhE8yLKysrCURekWpCysTIj2owZIzDH4pxOmTMfU1eat_3vgwmc%3D',
   'type': 'captcha_required'}],
 'request_id': '170594210943402dc7c0c36b7561740f'}

Соберем в одну таблицу.

In [30]:
info = []

for l in tqdm(links):
    v = requests.get(l)
    v_loaded = json.loads(v.text)
    
    if v_loaded['salary']['currency'] == 'RUR':
    
        descr = v_loaded['description']
        skills = [i['name'] for i in v_loaded['key_skills']]
        money_from = v_loaded['salary'].get('from', np.nan)
        money_to = v_loaded['salary'].get('to', np.nan)
        url = v_loaded['alternate_url']

        info.append([descr, skills, money_from, money_to, url])
        sleep(3)

 76%|█████████████████████████████████████████████████████████████                   | 113/148 [04:47<01:29,  2.55s/it]


TypeError: 'NoneType' object is not subscriptable

In [29]:
links[36]

'https://api.hh.ru/vacancies/91624035?host=hh.ru'

In [34]:
df = pd.DataFrame(info, columns=['description', 'key_skills', 'salary_from', 'salary_to', 'url'])

In [83]:
df

,description,key_skills,salary_from,salary_to,url
0,"<p>Мы IT-компания из Санкт-Петербурга, которая...","[Java SE, Spring Framework, PostgreSQL, ООП]","60,000.00","120,000.00",https://hh.ru/vacancy/89601541
1,<p>Мы создали<strong> RANKS.pro</strong> — Сис...,"[Python, Математическая статистика, SQL, Panda...","50,000.00","100,000.00",https://hh.ru/vacancy/89380163
2,<p><em><strong>IDF Eurasia </strong></em>специ...,[],"40,000.00",NaN,https://hh.ru/vacancy/87623047
3,<p><strong>Karpov.courses </strong>— молодая E...,"[Документооборот, Работа с большим объемом инф...","50,000.00",NaN,https://hh.ru/vacancy/89215196
4,"<strong>O2 Trading - ищем, проверяем, внедряем...","[Python, SQL, Анализ данных, Работа с большим ...","40,000.00","100,000.00",https://hh.ru/vacancy/89072255
...,...,...,...,...,...
117,<p>Pygmalion AI — это передовая AI-платформа д...,"[Python, Аналитическое мышление, Data Analysis...","400,000.00",NaN,https://hh.ru/vacancy/88412207
118,<p>Pygmalion AI — это передовая AI-платформа д...,"[Python, Аналитическое мышление, Data Analysis...","400,000.00",NaN,https://hh.ru/vacancy/88412212
119,<p><strong>Про нас:</strong></p> <p>OTUS сегод...,"[Расчет заработной платы, Расчет зарплаты, Бух...",NaN,"80,000.00",https://hh.ru/vacancy/89142460
120,<p>Pygmalion AI — это передовая AI-платформа д...,"[Python, Аналитическое мышление, Data Analysis...","400,000.00",NaN,https://hh.ru/vacancy/88412209


In [84]:
df['key_skills'].sum()

['Java SE',
 'Spring Framework',
 'PostgreSQL',
 'ООП',
 'Python',
 'Математическая статистика',
 'SQL',
 'Pandas',
 'Машинное обучение',
 'Документооборот',
 'Работа с большим объемом информации',
 'Деловая переписка',
 'Грамотная речь',
 'amoCRM',
 'Делопроизводство',
 'Электронный документооборот',
 'Python',
 'SQL',
 'Анализ данных',
 'Работа с большим объемом информации',
 'Базы данных',
 'Аналитическое мышление',
 'Работа с базами данных',
 'Математическая статистика',
 'Python',
 'PostgreSQL',
 'Docker',
 'SQL',
 'Git',
 'FastAPI',
 'Backend',
 'MS SQL',
 'Базы данных',
 'ООП',
 'SOLID',
 'Работа в команде',
 'Грамотная речь',
 'Телефонные переговоры',
 'Работа с базами данных',
 'Ориентация на результат',
 'Git',
 'Java Core',
 'Spring',
 'Java',
 'Hibernate',
 'Apache Maven',
 'PostgreSQL',
 'Hibernate ORM',
 'Web',
 'MySQL',
 'MongoDB',
 'Linux',
 'ООП',
 'Teamplayer',
 'Time management',
 'Negotiation skills',
 'Analytical skills',
 'Sales Skills',
 'JavaScript',
 'Bootstrap

In [85]:
Counter(df['key_skills'].sum()).most_common(15)

[('Python', 37),
 ('SQL', 27),
 ('Аналитическое мышление', 16),
 ('Data Analysis', 12),
 ('Анализ данных', 11),
 ('Работа с базами данных', 11),
 ('ETL', 11),
 ('MySQL', 10),
 ('Git', 9),
 ('Английский язык', 9),
 ('Бизнес-анализ', 9),
 ('Agile Project Management', 9),
 ('Грамотная речь', 8),
 ('Big Data', 8),
 ('NLP', 8)]

In [86]:
pd.options.display.float_format = '{:,.2f}'.format

df[['salary_from', 'salary_to']].describe()

,salary_from,salary_to
count,106.00,63.00
mean,"150,718.40","234,171.43"
std,"122,098.93","166,547.08"
min,"30,000.00","30,000.00"
25%,"50,000.00","100,000.00"
50%,"100,000.00","150,000.00"
75%,"200,000.00","390,000.00"
max,"400,000.00","550,000.00"


In [87]:
with pd.option_context('display.max_colwidth', None):
    display(df[df['salary_from'] == df['salary_from'].max()])

description  \
68                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     <p>Pygmalion AI — это передовая AI-платформа для создания, публикации и продвижения контента в социальных медиа. Для достижения этой цели мы ищем Старшего инженера по данным, который будет играть ключевую роль во внедрении практик DataOps для эффективного управления данными социальных медиа. В этой роли подразумевается контроль за всем жизненным циклом данных, обеспечение целостности данных и расширение возможностей нашей AI-платформы для обеспечения исключительного пользовательского опыта.</p> <p>Ключевые обязанности:</p> <ul> <li>Проектирование, разработка и поддержание надежных и масштабируемых конвейеров данных для сбора, обработки и хранения данных из различных источников социальных медиа и пользовательских взаимодействий.</li> <li>Проектирование хранилища данных.</li> <li>Реализация строгих проверок качества данных и процессов валидации для поддержания целостности, точности и надежности данных социальных медиа, используемых нашими AI-моделями.</li> <li>Автоматизация процессов извлечения, трансформации и загрузки (ETL) для оптимизации загрузки и трансформации данных, уменьшения ручного вмешательства и повышения эффективности.</li> <li>Непрерывный мониторинг и оптимизация конвейеров данных для улучшения скорости, надежности и масштабируемости, обеспечивая бесшовную работу нашего AI-ассистента.</li> <li>Тесное сотрудничество с инженерами машинного обучения и межфункциональными командами для понимания требований к данным и предоставления необходимой инфраструктуры данных для разработки и обучения моделей.</li> <li>Проведение практик управления данными, гарантирование конфиденциальности данных, безопасности и соответствия соответствующим нормативам, включая GDPR, в контексте данных социальных медиа.</li> <li>Установление показателей производительности и внедрение решений мониторинга для выявления и устранения узких мест или аномалий в конвейере данных.</li> <li>Сотрудничество с аналитиками данных и бизнес-командами для проектирования интерактивных панелей инструментов, которые обеспечивают принятие решений на основе данных.</li> <li>Разработка и поддержка датамартов и панелей инструментов, предоставляющих оперативные аналитические данные по данным социальных медиа.</li> <li>Быть в курсе появляющихся технологий, инструментов и фреймворков в области данных, оценивая их потенциал для улучшения процессов инженерии данных.</li> </ul> <p> </p> <p>Требования к кандидату:</p> <ul> <li>Степень бакалавра или магистра по компьютерным наукам, инженерии данных или смежной области.</li> <li>Доказанный опыт работы в области инженерии данных, с акцентом на процессы ETL, разработку конвейеров данных и обеспечение качества данных.</li> <li>Высокая профессиональная компетенция в языках программирования, таких как Python, SQL, а также знание библиотек и фреймворков инженерии данных.</li> <li>Опыт работы с облачными решениями для хранения и обработки данных, такими как AWS,

## Задание 1

Найдите все вакансии, в которых тип занятости "Волонтер".
В этом вам поможет поле "employment", в которое подставляются значения из общих словарей: https://api.hh.ru/dictionaries

In [12]:
r = requests.get('https://api.hh.ru/dictionaries')
hh_dict = json.loads(r.text)
#print(hh_dict['vacancy_search_fields'])
for i in hh_dict:
    print(i)

[{'id': 'name', 'name': 'в названии вакансии'}, {'id': 'company_name', 'name': 'в названии компании'}, {'id': 'description', 'name': 'в описании вакансии'}]
vacancy_label
resume_access_type
vacancy_search_order
vacancy_search_fields
gender
preferred_contact_type
travel_time
relocation_type
business_trip_readiness
resume_contacts_site_type
employer_type
employer_relation
negotiations_state
applicant_negotiation_status
negotiations_participant_type
negotiations_order
resume_moderation_note
vacancy_relation
resume_status
resume_search_logic
resume_search_fields
messaging_status
employer_active_vacancies_order
employer_archived_vacancies_order
employer_hidden_vacancies_order
applicant_comments_order
vacancy_not_prolonged_reason
resume_hidden_fields
phone_call_status
experience
employment
schedule
education_level
currency
vacancy_billing_type
applicant_comment_access_type
vacancy_cluster
driver_license_types
language_level
working_days
working_time_intervals
working_time_modes
vacancy_type


In [81]:
#hh_dict['employment']
hh_dict['employer']


KeyError: 'employer_id'

In [47]:
info = []

for l in tqdm(links):
    v = requests.get(l)
    v_loaded = json.loads(v.text)
    
    if v_loaded['employment']['id'] == 'volunteer':
    
        descr = v_loaded['description']
        skills = [i['name'] for i in v_loaded['key_skills']]
        money_from = v_loaded['salary'].get('from', np.nan)
        money_to = v_loaded['salary'].get('to', np.nan)
        url = v_loaded['alternate_url']

        info.append([descr, skills, money_from, money_to, url])
        sleep(3)


 81%|████████████████████████████████████████████████████████████████▊               | 120/148 [00:14<00:03,  8.06it/s]


KeyError: 'employment'

In [3]:
v = requests.get('https://api.hh.ru/vacancies?employment=volunteer&per_page=100')
volunteer = json.loads(v.text)['items']
volunteer

[{'id': '91191785',
  'premium': False,
  'name': 'Менеджер киберспортивных мероприятий',
  'department': None,
  'has_test': False,
  'response_letter_required': True,
  'area': {'id': '2',
   'name': 'Санкт-Петербург',
   'url': 'https://api.hh.ru/areas/2'},
  'salary': None,
  'type': {'id': 'open', 'name': 'Открытая'},
  'address': None,
  'response_url': None,
  'sort_point_distance': None,
  'published_at': '2024-01-12T00:25:32+0300',
  'created_at': '2024-01-12T00:25:32+0300',
  'archived': False,
  'apply_alternate_url': 'https://hh.ru/applicant/vacancy_response?vacancyId=91191785',
  'show_logo_in_search': None,
  'insider_interview': None,
  'url': 'https://api.hh.ru/vacancies/91191785?host=hh.ru',
  'alternate_url': 'https://hh.ru/vacancy/91191785',
  'relations': [],
  'employer': {'id': '4651728',
   'name': 'Ростройгазпром',
   'url': 'https://api.hh.ru/employers/4651728',
   'alternate_url': 'https://hh.ru/employer/4651728',
   'logo_urls': {'240': 'https://hhcdn.ru/empl

In [71]:
vol = volunteer[0]
vol['name']
vol['employment'] == ''

False

In [75]:
cnt = 0
info2 = []
for i in volunteer:
    try:
        name = i['name']
    except:
        name = np.nan
    try:
        money_from = i['salary'].get('from', np.nan)
    except:
        money_from = np.nan
    try:
        money_to = i['salary'].get('to', np.nan)
    except:
        money_to = np.nan
        
    try:
        curr = i['salary'].get('currency', np.nan)
    except:
        curr = np.nan
    
    try:
        area = i['area'].get('name', np.nan)
    except:
        area = np.nan
        
    try:
        emp = i['employer'].get('name', np.nan)
    except:
        emp = np.nan
    url = i['alternate_url']


    info2.append([name, money_from, money_to, curr, area, emp, url])
    
    cnt += 1
    print(cnt)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


In [80]:
df_v = pd.DataFrame(info2, columns = ['name', 'money_from', 'money_to', 'curr', 'area', 'emp', 'url'])
df_v.sort_values('money_from', ascending = False)

,name,money_from,money_to,curr,area,emp,url
34,Менеджер по работе с недвижимостью,300000.0,NaN,RUR,Сочи,ОНИКС-Недвижимость,https://hh.ru/vacancy/91949166
26,Помощник юриста,250000.0,300000.0,KZT,Алматы,FINTEND,https://hh.ru/vacancy/90278576
28,Промоутер-распространитель,100000.0,200000.0,KZT,Шымкент,Омыртқа орталығы #1,https://hh.ru/vacancy/91066920
18,Менеджер по B2B по работе с партнерами (удаленка),70000.0,200000.0,RUR,Москва,Мехдиев Илкин,https://hh.ru/vacancy/91680520
21,Администратор салона (Королев),65000.0,NaN,RUR,Королев,СПА СИАМ,https://hh.ru/vacancy/90975616
10,Оператор в отдел заявок,55000.0,NaN,RUR,Самара,ЮК Заря,https://hh.ru/vacancy/89822243
1,Волонтер в дом-интернат,50000.0,NaN,RUR,Омск,Екатерининский Екатерининский Дом-Интернат Име...,https://hh.ru/vacancy/91619818
33,Мойщик автомобилей в дилерский центр Skoda,45000.0,NaN,RUR,Рязань,Группа Компаний Автоимпорт,https://hh.ru/vacancy/89055725
0,"Учитель истории России, всеобщей истории, обще...",40000.0,85000.0,RUR,Москва,Казатель,https://hh.ru/vacancy/91945968
24,Учитель химии,40000.0,85000.0,RUR,Москва,Казатель,https://hh.ru/vacancy/91945936


In [55]:
print(volunteer[0]['name'])
print(volunteer[0]['salary']['from'])
print(volunteer[0]['salary']['to'])
print(volunteer[0]['salary']['currency'])
print(volunteer[0]['area']['name'])
print(volunteer[0]['employer']['name'])
print(volunteer[0]['alternate_url'])

Учитель истории России, всеобщей истории, обществознания
40000
85000
RUR
Москва
Казатель
https://hh.ru/vacancy/91945968


In [37]:
df = pd.DataFrame(info, columns=['description', 'key_skills', 'salary_from', 'salary_to', 'url'])


[['<p>В команду аналитиков требуется кандидат, умеющий быстро и точно анализировать большие объемы данных.</p> <p><strong>Вам предстоит заниматься:</strong></p> <ul> <li>Анализом данных с помощью Python - Ad Hoc аналитикой</li> <li>Подготовкой данных и обучением моделей ML</li> <li>Подготовкой ТЗ для команды разработчиков</li> <li>Визуализацией данных, созданием презентаций и отчетности</li> </ul> <strong>Требования:</strong> <ul> <li>Знание Python (pandas, numpy, sklearn, seaborn, matploltlib, plotly)</li> <li>Знание SQL (умение создавать таблицы и выводить данные)</li> <li>Высшее образование (математическое/информационные технологии)</li> <li>Знание ML будут плюсом.</li> </ul> <strong>Условия:</strong> <ul> <li>Стабильный оклад</li> <li>Официальное оформление по ТК РФ с первого дня</li> <li>График работы: 5/2, с 10:00 до 18:30. Суббота, воскресенье-выходные</li> <li>Отдых летом на море и зимой на горнолыжных курортах при финансовой поддержке от группы компаний</li> <li>Компенсация ра

## Задание 2

Найдите, сколько вакансий на hh было опубликовано за эту неделю.

In [ ]:
https://api.hh.ru/vacancies?period=8&area=113
    
https://api.hh.ru/vacancies?area=113&date_from=2024-01-15&date_to=2024-01-18

## Задание 3

Найдите все вакансии, которые сейчас открыты у вашей компании (учебного заведения / интересующей вас компании). Загрузите данные в таблицу и сделайте описательную статистику заработных плат в вакансиях с помощью метода describe().

В этом вам поможет модуль "Поиск работодателя": https://api.hh.ru/openapi/redoc#tag/Rabotodatel/operation/search-employer

In [104]:
v = requests.get('https://api.hh.ru/vacancies?employer_id=4649269&per_page=1')
num = json.loads(v.text)['found']

vacancies = []

for i in range(0, num):
    vac = requests.get(f"https://api.hh.ru/vacancies?employer_id=4649269&per_page=1&page={num}")
    vacancies.extend(json.loads(vac.text)['items'])
    print(i)
    
print(vacancies)
#сделать без items, забрать по ключу found
#volunteer = json.loads(v.text)['items']
#volunteer

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116


KeyError: 'items'

In [99]:
cnt = 0
info2 = []
for i in volunteer:
    try:
        name = i['name']
    except:
        name = np.nan
    try:
        money_from = i['salary'].get('from', np.nan)
    except:
        money_from = np.nan
    try:
        money_to = i['salary'].get('to', np.nan)
    except:
        money_to = np.nan
        
    try:
        curr = i['salary'].get('currency', np.nan)
    except:
        curr = np.nan
    
    try:
        area = i['area'].get('name', np.nan)
    except:
        area = np.nan
        
    try:
        emp = i['employer'].get('name', np.nan)
    except:
        emp = np.nan
    url = i['alternate_url']


    info2.append([name, money_from, money_to, curr, area, emp, url])
    
    cnt += 1
    print(cnt)

1
